<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/crosscurrencyratehelpers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import QuantLib as ql
import unittest

# Helper to create a flat forward rate curve easily
def flat_rate_py(evaluation_date, forward_rate, day_counter):
    return ql.FlatForward(evaluation_date, ql.QuoteHandle(ql.SimpleQuote(forward_rate)), day_counter)

class XccyTestDatum:
    def __init__(self, n, units, basis):
        self.n = n
        self.units = units
        self.basis = basis

class CommonVars:
    def __init__(self):
        self.curve_settlement_days = 0
        self.instrument_settlement_days = 2
        self.business_convention = ql.Following
        self.calendar = ql.TARGET()
        self.day_count = ql.Actual365Fixed()
        self.end_of_month = False

        self.basis_point = 1.0e-4
        self.fx_spot = 1.25

        self.base_ccy_idx_handle = ql.RelinkableYieldTermStructureHandle()
        self.quote_ccy_idx_handle = ql.RelinkableYieldTermStructureHandle()

        self.base_ccy_idx = ql.Euribor3M(self.base_ccy_idx_handle)
        self.quote_ccy_idx = ql.USDLibor(ql.Period(3, ql.Months), self.quote_ccy_idx_handle)

        self.basis_data = [
            XccyTestDatum(1, ql.Years, -14.5), XccyTestDatum(18, ql.Months, -18.5),
            XccyTestDatum(2, ql.Years, -20.5), XccyTestDatum(3, ql.Years, -23.75),
            XccyTestDatum(4, ql.Years, -25.5), XccyTestDatum(5, ql.Years, -26.5),
            XccyTestDatum(7, ql.Years, -26.75), XccyTestDatum(10, ql.Years, -26.25),
            XccyTestDatum(15, ql.Years, -24.75), XccyTestDatum(20, ql.Years, -23.25),
            XccyTestDatum(30, ql.Years, -20.50)
        ]

        self.today = self.calendar.adjust(ql.Date(6, ql.September, 2013))
        # ql.Settings.instance().evaluationDate = self.today # Set in test methods to avoid global side effects

        self.instrument_settlement_dt = self.calendar.advance(self.today, self.instrument_settlement_days, ql.Days)
        self.curve_settlement_dt = self.calendar.advance(self.today, self.curve_settlement_days, ql.Days)

        self.base_ccy_idx_handle.linkTo(flat_rate_py(self.curve_settlement_dt, 0.007, self.day_count))
        self.quote_ccy_idx_handle.linkTo(flat_rate_py(self.curve_settlement_dt, 0.015, self.day_count))

    def constant_notional_xccy_rate_helper(self, q_datum, collateral_handle,
                                           is_fx_base_currency_collateral_currency,
                                           is_basis_on_fx_base_currency_leg):
        quote_handle = ql.QuoteHandle(ql.SimpleQuote(q_datum.basis * self.basis_point))
        tenor = ql.Period(q_datum.n, q_datum.units)
        return ql.ConstNotionalCrossCurrencyBasisSwapRateHelper(
            quote_handle, tenor, self.instrument_settlement_days, self.calendar,
            self.business_convention, self.end_of_month, self.base_ccy_idx, self.quote_ccy_idx,
            collateral_handle, is_fx_base_currency_collateral_currency,
            is_basis_on_fx_base_currency_leg)

    def build_constant_notional_xccy_rate_helpers(self, xccy_data, collateral_handle,
                                                  is_fx_base_currency_collateral_currency,
                                                  is_basis_on_fx_base_currency_leg):
        instruments = []
        for item in xccy_data:
            instruments.append(self.constant_notional_xccy_rate_helper(
                item, collateral_handle, is_fx_base_currency_collateral_currency,
                is_basis_on_fx_base_currency_leg))
        return instruments

    def resetting_xccy_rate_helper(self, q_datum, collateral_handle,
                                   is_fx_base_currency_collateral_currency,
                                   is_basis_on_fx_base_currency_leg,
                                   is_fx_base_currency_leg_resettable):
        quote_handle = ql.QuoteHandle(ql.SimpleQuote(q_datum.basis * self.basis_point))
        tenor = ql.Period(q_datum.n, q_datum.units)
        return ql.MtMCrossCurrencyBasisSwapRateHelper(
            quote_handle, tenor, self.instrument_settlement_days, self.calendar,
            self.business_convention, self.end_of_month, self.base_ccy_idx, self.quote_ccy_idx,
            collateral_handle, is_fx_base_currency_collateral_currency,
            is_basis_on_fx_base_currency_leg, is_fx_base_currency_leg_resettable)

    def build_resetting_xccy_rate_helpers(self, xccy_data, collateral_handle,
                                          is_fx_base_currency_collateral_currency,
                                          is_basis_on_fx_base_currency_leg,
                                          is_fx_base_currency_leg_resettable):
        instruments = []
        for item in xccy_data:
            instruments.append(self.resetting_xccy_rate_helper(
                item, collateral_handle, is_fx_base_currency_collateral_currency,
                is_basis_on_fx_base_currency_leg, is_fx_base_currency_leg_resettable))
        return instruments

    def leg_schedule(self, tenor_period, idx):
        return ql.MakeSchedule().from_(self.instrument_settlement_dt) \
                                .to(self.instrument_settlement_dt + tenor_period) \
                                .withTenor(idx.tenor()) \
                                .withCalendar(self.calendar) \
                                .withConvention(self.business_convention) \
                                .endOfMonth(self.end_of_month) \
                                .backwards().makeSchedule()

    def constant_notional_leg(self, schedule, idx, notional, basis):
        leg = ql.IborLeg([notional], schedule, idx, spreads=[basis]) # spreads as list

        # Explicitly add notional exchanges
        # Python's IborLeg might not add these by default as in some C++ helper contexts
        if leg: # ensure leg is not empty
            initial_payment_date = ql.CashFlows.startDate(leg)
            leg.append(ql.SimpleCashFlow(-notional, initial_payment_date))
            last_payment_date = ql.CashFlows.maturityDate(leg)
            leg.append(ql.SimpleCashFlow(notional, last_payment_date))
        return leg


    def build_xccy_basis_swap_proxy(self, q_datum, fx_spot_val,
                                    is_fx_base_currency_collateral_currency, # Unused in this proxy build
                                    is_basis_on_fx_base_currency_leg):
        base_ccy_leg_notional = 1.0
        quote_ccy_leg_notional = base_ccy_leg_notional * fx_spot_val

        base_ccy_leg_basis = q_datum.basis * self.basis_point if is_basis_on_fx_base_currency_leg else 0.0
        quote_ccy_leg_basis = 0.0 if is_basis_on_fx_base_currency_leg else q_datum.basis * self.basis_point

        swaps = []
        payer_flag = True # Arbitrary, will be !payer for one leg

        base_ccy_leg = self.constant_notional_leg(
            self.leg_schedule(ql.Period(q_datum.n, q_datum.units), self.base_ccy_idx),
            self.base_ccy_idx, base_ccy_leg_notional, base_ccy_leg_basis)
        swaps.append(ql.Swap([base_ccy_leg], [not payer_flag]))

        quote_ccy_leg = self.constant_notional_leg(
            self.leg_schedule(ql.Period(q_datum.n, q_datum.units), self.quote_ccy_idx),
            self.quote_ccy_idx, quote_ccy_leg_notional, quote_ccy_leg_basis)
        swaps.append(ql.Swap([quote_ccy_leg], [payer_flag]))

        return swaps


class CrossCurrencyRateHelpersTests(unittest.TestCase):

    def setUp(self):
        self.vars = CommonVars()
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        ql.Settings.instance().evaluationDate = self.vars.today

    def tearDown(self):
        ql.Settings.instance().evaluationDate = self.saved_eval_date

    def _test_constant_notional_xccy_swaps_npv(self, is_fx_base_currency_collateral_currency,
                                               is_basis_on_fx_base_currency_leg):
        collateral_handle = self.vars.base_ccy_idx_handle if is_fx_base_currency_collateral_currency \
                            else self.vars.quote_ccy_idx_handle

        collateral_ccy_leg_engine = ql.DiscountingSwapEngine(collateral_handle)

        instruments = self.vars.build_constant_notional_xccy_rate_helpers(
            self.vars.basis_data, collateral_handle,
            is_fx_base_currency_collateral_currency, is_basis_on_fx_base_currency_leg)

        foreign_ccy_curve = ql.PiecewiseYieldCurveDiscountLogLinear( # Specific class for Discount, LogLinear
            self.vars.curve_settlement_dt, instruments, self.vars.day_count)
        foreign_ccy_curve.enableExtrapolation()
        foreign_ccy_handle = ql.YieldTermStructureHandle(foreign_ccy_curve)
        foreign_ccy_leg_engine = ql.DiscountingSwapEngine(foreign_ccy_handle)

        tolerance = 1.0e-12

        for i in range(len(self.vars.basis_data)):
            quote = self.vars.basis_data[i]
            xccy_swap_proxy_legs = self.vars.build_xccy_basis_swap_proxy(
                quote, self.vars.fx_spot,
                is_fx_base_currency_collateral_currency, # Passed for completeness, not used by build_xccy_basis_swap_proxy
                is_basis_on_fx_base_currency_leg)

            if is_fx_base_currency_collateral_currency:
                xccy_swap_proxy_legs[0].setPricingEngine(collateral_ccy_leg_engine)
                xccy_swap_proxy_legs[1].setPricingEngine(foreign_ccy_leg_engine)
            else:
                xccy_swap_proxy_legs[0].setPricingEngine(foreign_ccy_leg_engine)
                xccy_swap_proxy_legs[1].setPricingEngine(collateral_ccy_leg_engine)

            p = ql.Period(quote.n, quote.units)
            base_ccy_leg_npv = self.vars.fx_spot * xccy_swap_proxy_legs[0].NPV()
            quote_ccy_leg_npv = xccy_swap_proxy_legs[1].NPV()
            npv = base_ccy_leg_npv + quote_ccy_leg_npv

            self.assertAlmostEqual(npv, 0.0, delta=tolerance,
                                   msg=f"Failed to price XCCY basis swap to par (NPV={npv:.5e})\n"
                                       f"Basis: {quote.basis}, Tenor: {p}")

    def _test_resetting_xccy_swaps_curves(self, is_fx_base_currency_collateral_currency,
                                          is_basis_on_fx_base_currency_leg,
                                          is_fx_base_currency_leg_resettable):
        collateral_handle = self.vars.base_ccy_idx_handle if is_fx_base_currency_collateral_currency \
                            else self.vars.quote_ccy_idx_handle

        resetting_instruments = self.vars.build_resetting_xccy_rate_helpers(
            self.vars.basis_data, collateral_handle, is_fx_base_currency_collateral_currency,
            is_basis_on_fx_base_currency_leg, is_fx_base_currency_leg_resettable)

        const_notional_instruments = self.vars.build_constant_notional_xccy_rate_helpers(
            self.vars.basis_data, collateral_handle, is_fx_base_currency_collateral_currency,
            is_basis_on_fx_base_currency_leg)

        resetting_curve = ql.PiecewiseYieldCurveDiscountLogLinear(
            self.vars.curve_settlement_dt, resetting_instruments, self.vars.day_count)
        resetting_curve.enableExtrapolation()

        const_notional_curve = ql.PiecewiseYieldCurveDiscountLogLinear(
            self.vars.curve_settlement_dt, const_notional_instruments, self.vars.day_count)
        const_notional_curve.enableExtrapolation()

        tolerance = 1.0e-4 * 5 # 5 bps

        for i in range(len(self.vars.basis_data)):
            # Use maturity date from the helper itself
            maturity_date = resetting_instruments[i].maturityDate()

            resetting_zero = resetting_curve.zeroRate(maturity_date, self.vars.day_count, ql.Continuous).rate()
            const_notional_zero = const_notional_curve.zeroRate(maturity_date, self.vars.day_count, ql.Continuous).rate()

            self.assertAlmostEqual(resetting_zero, const_notional_zero, delta=tolerance,
                                   msg=f"Too large difference between resetting and constant notional curve zero rates.\n"
                                       f"Resetting Zero: {resetting_zero:.5f}, Const Notional Zero: {const_notional_zero:.5f}\n"
                                       f"Maturity: {maturity_date}")

    # --- Test cases for constant notional swaps ---
    def test_const_notional_collateral_quote_basis_base(self):
        print("Testing constant notional: collateral in quote ccy, basis in base ccy...")
        self._test_constant_notional_xccy_swaps_npv(False, True)

    def test_const_notional_collateral_base_basis_quote(self):
        print("Testing constant notional: collateral in base ccy, basis in quote ccy...")
        self._test_constant_notional_xccy_swaps_npv(True, False)

    def test_const_notional_collateral_base_basis_base(self):
        print("Testing constant notional: collateral and basis in base ccy...")
        self._test_constant_notional_xccy_swaps_npv(True, True)

    def test_const_notional_collateral_quote_basis_quote(self):
        print("Testing constant notional: collateral and basis in quote ccy...")
        self._test_constant_notional_xccy_swaps_npv(False, False)

    # --- Test cases for resetting (MTM) swaps ---
    def test_resetting_collateral_quote_basis_base(self):
        print("Testing resetting: collateral in quote ccy, basis in base ccy (quote leg resettable)...")
        self._test_resetting_xccy_swaps_curves(False, True, False) # isFxBaseCurrencyLegResettable = False

    def test_resetting_collateral_base_basis_quote(self):
        print("Testing resetting: collateral in base ccy, basis in quote ccy (base leg resettable)...")
        self._test_resetting_xccy_swaps_curves(True, False, True) # isFxBaseCurrencyLegResettable = True

    def test_resetting_collateral_base_basis_base(self):
        print("Testing resetting: collateral and basis in base ccy (base leg resettable)...")
        self._test_resetting_xccy_swaps_curves(True, True, True)

    def test_resetting_collateral_quote_basis_quote(self):
        print("Testing resetting: collateral and basis in quote ccy (quote leg resettable)...")
        self._test_resetting_xccy_swaps_curves(False, False, False)

    def test_exception_when_instrument_tenor_shorter_than_index_frequency(self):
        print("Testing exception when instrument tenor is shorter than index frequency...")
        data = [XccyTestDatum(1, ql.Months, 10.0)] # 1M tenor, 3M index
        collateral_handle = ql.YieldTermStructureHandle() # Dummy handle for this test

        with self.assertRaisesRegex(RuntimeError, "swap tenor \\(1M\\) is shorter than index frequency \\(3M\\)"):
             self.vars.build_constant_notional_xccy_rate_helpers(
                 data, collateral_handle, True, True)

        # Also test for MtMCrossCurrencyBasisSwapRateHelper
        with self.assertRaisesRegex(RuntimeError, "swap tenor \\(1M\\) is shorter than index frequency \\(3M\\)"):
            self.vars.build_resetting_xccy_rate_helpers(
                data, collateral_handle, True, True, True)


if __name__ == '__main__':
    print("Testing QuantLib " + ql.__version__)
    unittest.main(argv=['first-arg-is-ignored'], exit=False)